In [ ]:
!git clone https://github.com/omriallouche/text_classification_from_zero_to_hero.git --depth 1
import os
os.chdir('text_classification_from_zero_to_hero/notebooks')

# Contextual Embeddings with ELMo
## Part 3 of the Workshop "Text Classification - From Zero to Hero", by Dr. Omri Allouche, Gong.io, Bar Ilan University

ELMo learns contextualized word vectors by running the text through a deep recurrent network.  
ELMo is actually an algorithm for unsupervised learning and does not make any use of the labels we have for our text classification task. The authors do show that contextualized word vectors obtained using ELMo increase text classification performance in a large array of tasks. Let's see if we see a significant gain in our case!  

There are good examples of using ELMo in both [the AllenNLP github repo](https://github.com/allenai/allennlp/blob/master/tutorials/how_to/elmo.md) and [this AnalyticsVidhya post](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/?utm_source=blog&utm_medium=top-pretrained-models-nlp-article). In this guide we'll use the python package [Flair](https://github.com/zalandoresearch/flair) to get ELMo embeddings.

## Contextual word embeddings with ELMo in Flair
In Flair, you init a `Sentence` object given the tokens seperated by spaces.  
Sentence has a few useful attributes and methods

In [ ]:
!pip install allennlp
!pip install flair

In [21]:
from flair.embeddings import Sentence
sentence = Sentence('The grass is green .')

We also init a class of the desired embedding method. The `embed` method of this class gets a Sentence and adds to its tokens the relevant embedding.

In [22]:
from flair.embeddings import ELMoEmbeddings

# init embedding
elmo_embedding = ELMoEmbeddings()

elmo_embedding.embed(sentence)
for token in sentence:
    print(token)
    print(token.embedding.shape)
    print(token.embedding)

Token: 1 The
torch.Size([3072])
tensor([-0.3288,  0.2022, -0.5940,  ..., -1.2773,  0.3049,  0.2150])
Token: 2 grass
torch.Size([3072])
tensor([ 0.2539, -0.2363,  0.5263,  ..., -0.7001,  0.8798,  1.4191])
Token: 3 is
torch.Size([3072])
tensor([ 0.1915,  0.2300, -0.2894,  ..., -0.3626,  1.9066,  1.4520])
Token: 4 green
torch.Size([3072])
tensor([ 0.1779,  0.1309, -0.1041,  ..., -0.1006,  1.6152,  0.3299])
Token: 5 .
torch.Size([3072])
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.0106, -0.0833,  0.0669])


**Try it yourself:** Now, compare the embeddings obtained using ELMo for the same word in different contexts. Are they equal or different?

### (optional) Word sense disambiguation using ELMo
**Try it yourself:** Let's also try to see how ELMo handles word sense disambiguation. Below are 6 sentences with 2 different meanings of the word `bank`. Try to see if ELMo vectors indeed separate the two meanings.

In [23]:
sentences = [
    "I was walking along the river bank",
    "I saw a toad near the east bank of the river",
    "We had a nice picnic by the bank",
    "I need to deposit money from the bank",
    "The bank branch is closed",
    "He started working at the bank"
]

## Sentence Embedding and Text Classification

### Classify documents using the average of contextual word vectors
**Try it yourself:** *Optional:* In previous sections we've built a classifier using the average of non-contextual word vectors. Now, try to use contextual word embeddings on our dataset. Use the average of these vectors and apply a classifier on it to obtain the predictions. Is the performance better than for non-contextual word vectors?

In [24]:
from sklearn import linear_model
from sklearn import metrics
clf = linear_model.LogisticRegression(C=1e5)

In [25]:
import numpy as np
def get_sentence_embedding(sentence):
    sentence = Sentence(sentence)
    elmo_embedding.embed(sentence)
    sentence_embedding = np.mean( [np.array(token.embedding) for token in sentence], axis=0)
    return sentence_embedding

In [26]:
# Load the training set
import pandas as pd
df = pd.read_csv('../data/train.csv')
# Compute sentence embedding for the dataset
vectors = np.array([get_sentence_embedding(x) for x in df['text']])
y_truth = df['label']
clf.fit(vectors, y_truth)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [27]:
# Load the test set
df = pd.read_csv('../data/val.csv')
# Compute sentence embedding for the dataset
vectors = np.array([get_sentence_embedding(x) for x in df['text']])
y_truth = df['label']
y_predict = clf.predict(vectors)
metrics.accuracy_score(y_truth, y_predict, average='macro')

0.753968253968254

In [28]:
print(metrics.classification_report(y_truth, y_predict))

                       precision    recall  f1-score   support

   rec.sport.baseball       0.71      0.62      0.67        16
     rec.sport.hockey       0.73      0.80      0.76        20
   talk.politics.guns       0.83      0.86      0.84        22
talk.politics.mideast       0.76      0.72      0.74        18

            micro avg       0.76      0.76      0.76        76
            macro avg       0.76      0.75      0.75        76
         weighted avg       0.76      0.76      0.76        76

